In [ ]:
# import settings and functions
%run ./../../imports.ipynb

In [ ]:
nonlinear_a = 1
nonlinear_b = 1
nonlinear_c = 1.0

a = 134.0
b = -0.1047
c = 3.719 *10**(-5)
nonlinear_a = a
nonlinear_b = b
nonlinear_c = c

add_to_csv = 0
scaling = 0

run_analysis = False
run_test = False
my_nparts = 1

val_scale_scale = 0.1

mesh_name = "square_circle"
# params.mesh_file = "square_circle"

labels = False
labels = True

In [ ]:
# # creating custom colormap
# from matplotlib.colors import LinearSegmentedColormap
# # Create a custom colormap (blue to red without white)
# colors = [(0, 0, 1), (1, 0, 0)]  # Blue to Red
# n_bins = 100
# cmap_name = 'blue_to_red'
# custom_cmap_temperature = LinearSegmentedColormap.from_list(cmap_name, colors, N=n_bins)

# colors = ['green', 'yellow', 'magenta']
# cmap_name = 'green_to_yellow'
# custom_cmap_flux = LinearSegmentedColormap.from_list(cmap_name, colors, N=n_bins)

# custom_colorscale_temperature = [
#     [0, 'blue'],
#     [1, 'red']
# ]

# custom_colorscale_flux = [
#     [0, 'green'],
#     [0.5, 'yellow'],
#     [1, 'magenta']
# ]

# custom_colorscale_grad = [
#     [0, 'orange'],
#     [0.5, 'yellow'],
#     [1, 'purple']
# ]

In [ ]:
tools_dir = "~/um_view_release/bin"
# diffusion_nonlinear_graphite = "~/um_view/mofem_data_driven_finite_elements/diffusion_nonlinear_graphite"


In [ ]:
# import numpy as np
# import plotly as Plotly
# import plotly.graph_objects as go
# import pandas as pd
# import matplotlib.pyplot as plt
# # plt.style.use('ggplot')
# %matplotlib inline
# plt.rcParams['font.family'] = "Serif"
# # plt.rcParams['font.family'] = "Times New Roman"
# plt.rcParams['font.size'] = 15
# FS = 15

In [ ]:
class AttrDict(dict):
    def __getattr__(self, attr):
        if attr in self:
            return self[attr]
        raise AttributeError(f"'AttrDict' object has no attribute '{attr}'")

def generate_config(params):
    # Open the file for writing
    with open(params.config_name, 'w') as f:
        # FIX_ALL boundary condition (do not change)
        data = ['[SET_ATTR_PRESSURE_UNIFORM_OUT]', 'number_of_attributes=1', 'user1='+str(params.p_out)]
        # Use a for loop to write each line of data to the file
        for line in data:
            f.write(line + '\n')
            # print the data as it is written to the file
            print(line)
        data = ['[SET_ATTR_PRESSURE_UNIFORM_IN]', 'number_of_attributes=1', 'user1='+str(params.p_in)]
        # Use a for loop to write each line of data to the file
        for line in data:
            f.write(line + '\n')
            # print the data
            print(line)
            
        
def apply_congig(params):
    !read_med -med_file {params.med_file} -output_file {params.h5m_file} -meshsets_config {params.config_name} -dim 2 -adj_dim 1
    
def partition(params):
    !mofem_part -my_file {params.h5m_file} -output_file {params.part_out} -my_nparts {params.my_nparts} -dim 2 -adj_dim 1
   
# def run_diffusion_nonlinear_graphite(params):
#     !{diffusion_nonlinear_graphite} -file_name {params.part_out} -dt_order 2 -nonlinear_a {nonlinear_a} -nonlinear_b {nonlinear_b} -nonlinear_c {nonlinear_c} -scaling 0 -add_to_csv {add_to_csv}
        
    

In [ ]:
# params = AttrDict() # Attribute dictionary for storing the parameters
params.p_out = 0
params.p_in = 0
params.med_file = "square_circle.med"
params.h5m_file = "square_circle.h5m"
params.mesh_file = "square_circle"
params.part_out = "parts_out.h5m"
params.my_nparts = 1
params.exe = "~/um_view_release/mofem_data_driven_finite_elements/diffusion_nonlinear_graphite"
params.config_name = "bc.cfg"
params.config_file = "bc.cfg"


In [ ]:
# start display for showing results
display = Display(backend="xvfb", visible=False, size=(1024, 768))
display.start()

In [ ]:
if run_test:
    T_max = 1500
    T_min = 500
    add_to_csv = 0
    how_many = 10

    T_in = 500
    T_out = 1500

    params.p_out = T_out
    params.p_in = T_in
    generate_config(params)
    params.show_mesh = True
    # readMed(params)
    apply_congig(params)
    partition(params)

    print(params.part_out)

    !rm out*

    !{diffusion_nonlinear_graphite} -file_name {params.part_out} -my_order 2 -nonlinear_a {nonlinear_a} -nonlinear_b {nonlinear_b} -nonlinear_c {nonlinear_c} -scaling 0 -add_to_csv {add_to_csv}

    !convert.py out*

    params.show_mesh = False

In [ ]:
# if run_test:
# params.show_file = "out_iteration_"
params.show_file = "out_result_"
params.show_field = "T"
# params.show_file = "out_ori_result"
# params.show_field = "P_reference"
# params.warp_factor = 0.4  # warp factor
params.p_cmap=custom_cmap_temperature
params.clim=[500, 1500]
params.show_edges = True
params.p_save = "c5_exper_test_T.pdf"
show_results(params)

In [ ]:
T_max = 1500
T_min = 500
add_to_csv = 0
how_many = 10

if run_analysis:
    for T_in in np.linspace(T_min, T_max, how_many):
        for T_out in np.linspace(T_min, T_max, how_many):
            params.p_out = T_out
            params.p_in = T_in
            generate_config(params)
            apply_congig(params)
            partition(params)
            !{diffusion_nonlinear_graphite} -file_name {params.part_out} -dt_order 2 -nonlinear_a {nonlinear_a} -nonlinear_b {nonlinear_b} -nonlinear_c {nonlinear_c} -scaling 0 -add_to_csv {add_to_csv}
            add_to_csv = 1


In [ ]:
# data = pd.read_csv('data_VGQ_generated_used.csv', index_col=False)
fara = pd.read_csv('far_points.csv', index_col=False)
headers = []
headers = fara.columns.values
print(headers)


In [ ]:
fara = pd.read_csv('far_points.csv', index_col=False)

fara['value'] = fara['value'] / 0.0031942320890739257

fara['fluxx'] = fara['fluxx'] / 0.0031942320890739257
fara['fluxy'] = fara['fluxy'] / 0.014607657489315909

fara['value']
fara['fluxx']
fara['fluxy']

In [ ]:
# data = pd.read_csv('data_VGQ_generated_used.csv', index_col=False)
data = pd.read_csv('data_VGQ_generated.csv', index_col=False)
headers = []
headers = data.columns.values
print(headers)


In [ ]:
data = data.drop(data[data.index % 2 > 0].index)
data = data.drop(data[data.index % 3 > 0].index)


# data = data.drop(data[(data.gradx < -33)].index)
# data = data.drop(data[(data.gradx > 33)].index)
# data = data.drop(data[(data.grady < -33)].index)
# data = data.drop(data[(data.grady > 33)].index)


In [ ]:
fig, ax = plt.subplots()
ax.scatter(data["grady"], data["fluxy"], s=0.05)
ax.scatter(fara["grady"], fara["fluxy"], s=0.05, c='black')
# cbar = plt.colorbar()
# location='bottom'
# plt.ticklabel_format(style='sci', axis='both', scilimits=(0,0))
if labels:
    plt.xlabel('Gradient ' + r'$g_x$')
    plt.ylabel('Flux ' + r'$q_x$')
    # cbar.set_label('Temperature ' + r'$T$')
# plt.tight_layout()
ax.grid(True, ls=':')
x_limits = ax.get_xlim()
y_limits = ax.get_ylim()
plt.savefig('ana_dataset_temp_one_color.png', dpi=300, facecolor="none", edgecolor='none')
# plt.savefig('ana_dataset_temp.svg')
plt.show()

In [ ]:
fig, ax = plt.subplots()
# ax.scatter(data["grady"], data["fluxy"], s=0.05)
# # cbar = plt.colorbar()
# # location='bottom'
# # plt.ticklabel_format(style='sci', axis='both', scilimits=(0,0))
# if labels:
#     plt.xlabel('Gradient ' + r'$g_x$')
#     plt.ylabel('Flux ' + r'$q_x$')
#     # cbar.set_label('Temperature ' + r'$T$')
# plt.tight_layout()
ax.grid(True, ls=':')
ax.set_xlim(x_limits)
ax.set_ylim(y_limits)
plt.savefig('ana_dataset_temp_empty.png', dpi=300, facecolor="none", edgecolor='none')
# plt.savefig('ana_dataset_temp.svg')
plt.show()

In [ ]:
fig, ax = plt.subplots()
# plt.scatter(T, k, c=T, cmap=custom_cmap_temperature)
plt.scatter(data["grady"], data["fluxy"], c=data["value"], s=0.05, cmap=custom_cmap_temperature)
# plt.scatter(fara["grady"], fara["fluxy"], s=0.1, c='black')
cbar = plt.colorbar()
# location='bottom'
# plt.ticklabel_format(style='sci', axis='both', scilimits=(0,0))
if labels:
    plt.xlabel('Gradient ' + r'$g_x^*$')
    plt.ylabel('Flux ' + r'$q_x^*$')
    cbar.set_label('Temperature ' + r'$T^*$')
plt.tight_layout()
ax.grid(True, ls=':')
plt.savefig('ana_dataset_temp_coloured.png', dpi=300, facecolor="none", edgecolor='none')
plt.savefig('ana_dataset_temp_coloured.pdf')
plt.show()

In [ ]:
fig, ax = plt.subplots()
plt.scatter(data["grady"], data["fluxy"], c=data["value"], s=0.05, cmap=custom_cmap_temperature)
plt.scatter(fara["grady"], fara["fluxy"], s=0.1, c='black')
cbar = plt.colorbar()
# location='bottom'
# plt.ticklabel_format(style='sci', axis='both', scilimits=(0,0))
if labels:
    plt.xlabel('Gradient ' + r'$g_x$')
    plt.ylabel('Flux ' + r'$q_x$')
    cbar.set_label('Temperature ' + r'$T$')
# plt.tight_layout()
ax.grid(True, ls=':')
plt.savefig('ana_dataset_temp_narrower.png', dpi=300, facecolor="none", edgecolor='none')
# plt.savefig('ana_dataset_temp.svg')
plt.show()

In [ ]:
fig, ax = plt.subplots()
plt.scatter(data["value"], data["fluxy"], c=data["grady"], s=0.1, cmap="jet")
# plt.scatter(fara["value"], fara["fluxy"], s=0.1, color='black')
cbar = plt.colorbar()
# cbar.formatter.set_powerlimits((0, 0))
# plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
if labels: 
    plt.xlabel('Temperature ' + r'$T^*$')
    plt.ylabel('Flux ' + r'$q_x^*$')
    cbar.set_label('Gradient ' + r'$g_x^*$')
plt.tight_layout()
ax.grid(True, ls=':')
plt.savefig('ana_dataset_grad.png', dpi=300, facecolor="none", edgecolor='none')
plt.savefig('ana_dataset_grad.pdf')
plt.show()

In [ ]:
k = [-q/g for q,g in zip(data["fluxy"], data["grady"]) if abs(g) > 0.000001 ]

# g = [g for g in data["grady"] if abs(g) > 0.000001]
T = [T for T,g in zip(data["value"], data["grady"]) if abs(g) > 0.000001]
# q = [q for q,g in zip(data["fluxy"], data["grady"]) if abs(g) > 0.000001]

In [ ]:
# k = []
# for q_in, g_in in zip(data["fluxy"], data["grady"]):
#     k.append(-q_in/g_in)
    
# k = [-q/g for q,g in zip(data["fluxy"], data["grady"]) ]
fig, ax = plt.subplots()
plt.grid(True, ls=':')
plt.scatter(T, k, c=T, s=1, cmap=custom_cmap_temperature)

cbar = plt.colorbar()
if labels:
    plt.xlabel('Temperature ' + r'$T$')
    plt.ylabel('Conductivity ' + r'$k$')
    cbar.set_label('Temperature ' + r'$T$')
    
plt.tight_layout()
plt.savefig('ana_dataset_conductivity.png', dpi=300,facecolor="none", edgecolor='none')
plt.savefig('ana_dataset_conductivity.svg')
plt.show()

In [ ]:
# 3D scatter plot for G_x, G_y, and Q_y
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
sc = ax.scatter(data["grady"], data["value"], data["fluxy"], c=data["value"], s=1, cmap=custom_cmap_temperature)
cbar = fig.colorbar(sc, pad=0.2)
ax.set_xlabel(r'$g_y^*$', labelpad=10)
ax.set_ylabel(r'$T^*$', labelpad=10)
ax.set_zlabel(r'$q_y^*$', labelpad=10)
cbar.set_label(r'Temperature ' + r'$T^*$')
plt.tight_layout()
plt.savefig('dataset_3D_from_ana_loop.pdf')
plt.savefig('dataset_3D_from_ana_loop.png')
plt.show()

In [ ]:
# 3D scatter plot for G_x, G_y, and Q_y
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
sc = ax.scatter(data["grady"], data["gradx"], data["fluxy"], c=data["gradx"], s=1, cmap="jet")
cbar = fig.colorbar(sc, pad=0.2)
ax.set_xlabel(r'$g_y^*$', labelpad=10)
ax.set_ylabel(r'$T^*$', labelpad=10)
ax.set_zlabel(r'$q_y^*$', labelpad=10)
cbar.set_label(r'Temperature ' + r'$T^*$')
plt.tight_layout()
# plt.savefig('dataset_3D_from_ana_loop.pdf')
plt.show()

In [ ]:
import plotly as Plotly
import plotly.graph_objects as go
import plotly.io as py
fig = go.Figure()
fig.add_trace(go.Scatter3d(x=data["value"],
                           y=data["grady"],
                           z=data["fluxy"],
                           mode='markers',
                           marker={
                               'size': 2,
                               'opacity': 0.8,
                            #    'color': data["value"],
                               'color': '#66c2a5',
                            #    'colorbar_title': "Temperature",
                               'colorscale': "bluered"
                           },
                           ))

fig.add_trace(go.Scatter3d(x=fara["value"],
                           y=fara["grady"],
                           z=fara["fluxy"],
                           mode='markers',
                           marker={
                               'size': 2,
                               'opacity': 0.8,
                               'color': 'black'
                           },
                           ))

fig.update_layout(scene=dict(
    xaxis_title=r'Temperature',
    # xaxis_title='Gradient y',
    # xaxis_title_font_color='red',
    xaxis_title_font_size=20,
    yaxis_title=r'Gradient y',
    # yaxis_title_font_color='green',
    yaxis_title_font_size=20,
    zaxis_title=r'Flux y',
    # zaxis_title_font_color='blue',
    zaxis_title_font_size=20,

    ),
    font_family='Times New Roman',
    font_size=15,
    height=600,  # Adjust as needed
    margin=dict(r=20, b=10, l=10, t=10),
    # margin=dict(t=50, b=50, l=50, r=50),  # Adjust these values as needed
    scene_aspectmode='cube'
    )
# py.write_image(fig, 'figure.png')

fig.show()


In [ ]:
# 3D scatter plot for G_x, G_y, and Q_y
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
sc = ax.scatter(data["grady"], data["value"], data["fluxy"], s=0.05)
sc2 = ax.scatter(fara["grady"],fara["value"], fara["fluxy"], s=1, c='black')
ax.set_xlabel(r'$g_y^*$', labelpad=10)
ax.set_ylabel(r'$T^*$', labelpad=10)
ax.set_zlabel(r'$q_y^*$', labelpad=10)
plt.tight_layout()

# get the current axis limits
x_limits = ax.get_xlim()
y_limits = ax.get_ylim()
z_limits = ax.get_zlim()

plt.savefig('dataset_3D_from_ana_loop_0.png')
plt.show()

# Now, when you want to plot just one of them with the same limits:

fig2 = plt.figure()
ax2 = fig2.add_subplot(111, projection='3d')
sc3 = ax2.scatter(data["grady"], data["value"], data["fluxy"], s=0.05)
ax2.set_xlim(x_limits)
ax2.set_ylim(y_limits)
ax2.set_zlim(z_limits)
ax2.set_xlabel(r'$g_y^*$', labelpad=10)
ax2.set_ylabel(r'$T^*$', labelpad=10)
ax2.set_zlabel(r'$q_y^*$', labelpad=10)
plt.tight_layout()

plt.savefig('dataset_3D_from_ana_loop_1.png')
plt.show()

In [ ]:
import numpy as np
import plotly.graph_objects as go

# Create a surface plot
fig = go.Figure()

color1 = 'gray'

fig.add_trace(go.Scatter3d(x=data["value"],
                           y=data["fluxx"],
                           z=data["fluxy"],
                           mode='markers',
                           marker={
                               'size': 1,
                               'opacity': 0.8,
                               'color':data["gradx"],
                               'colorscale': "jet",
                               'colorbar':dict(thickness=20, title="Gradient x")
                           },
                           ))

fig.update_layout(
    scene=dict(
        xaxis_title='Temperature',
        # xaxis_title='Gradient y',
        xaxis_title_font_color='red',
        xaxis_title_font_size=20,
        yaxis_title='Flux x',
        yaxis_title_font_color='blue',
        yaxis_title_font_size=20,
        zaxis_title='Flux y',
        zaxis_title_font_color='blue',
        zaxis_title_font_size=20,
        # xaxis_title='Temperature',
        # yaxis_title='Flux x',
        # zaxis_title='Flux y',
        # camera=dict(
        #     eye=dict(x=1.5*eye_distance, y=1.5*eye_distance, z=0.8*eye_distance)
        # )
    ),
    font_family='Times New Roman',
    font_size=15,
    width=700,
    margin=dict(r=20, b=10, l=10, t=10),
    scene_aspectmode='cube'
)

fig.show()

# fig.write_image('surface_TQQ.png', scale=2)
# pio.write_image(fig, "surface_TQQ.svg", scale=2)

In [ ]:
# a = 134.0
# b = -0.1047
# c = 3.719 *10**(-5)

# T_min = 500
# T_max = 1500
# g_range = 1000

# T = []
# g = []
# q = []

# for T_in in np.linspace(T_min, T_max, 100):
#     for g_in in np.linspace(-g_range, g_range, 100):
#         T.append(T_in)
#         g.append(g_in)
#         q.append(-(a + b*T_in + c * T_in**2) * g_in)
        
# fig = go.Figure()
# fig.add_trace(go.Scatter3d(x=data["value"],
#                            y=data["grady"],
#                            z=data["fluxy"],
#                            mode='markers',
#                            marker={
#                                'size': 2,
#                                'opacity': 0.8,
# #                                'color': data["value"],
# #                                'colorbar_title': "Temperature",
# #                                'colorscale': "bluered"
#                            },
#                            ))

# fig.add_trace(go.Scatter3d(x=T,
#                            y=g,
#                            z=q,
#                            mode='markers',
#                            marker={
#                                'size': 2,
#                                'opacity': 0.8,
# #                                'color': T,
# #                                'colorbar_title': "Temperature",
# #                                'colorscale': "bluered"
#                            },
#                            ))

# fig.update_layout(scene=dict(
#     xaxis_title='Temperature',
#     # xaxis_title='Gradient y',
#     xaxis_title_font_color='red',
#     xaxis_title_font_size=20,
#     yaxis_title='Gradient y',
#     yaxis_title_font_color='green',
#     yaxis_title_font_size=20,
#     zaxis_title='Flux y',
#     zaxis_title_font_color='blue',
#     zaxis_title_font_size=20,

#     ),
#     font_family='Times New Roman',
#     font_size=15,
#     width=700,
#     margin=dict(r=20, b=10, l=10, t=10),
#     scene_aspectmode='cube'
#     )

# fig.show()

plot dataset grad grad flux

In [ ]:
max_vals = []

for header in headers:
    max_vals.append(data[header].max())

max_grad = 0
max_flux = 0
max_val = max_vals[4]

for i in [0,1]:
    if max_vals[i] > max_grad:
        max_grad = max_vals[i]
    if max_vals[i+2] > max_flux:
        max_flux = max_vals[i+2]

print(max_vals)
print(max_grad)
print(max_flux)
print(max_val)

scaling = [max_grad/max_flux, max_grad/max_val * val_scale_scale]

print(scaling)


In [ ]:
scal = np.genfromtxt('scaling.in')

scal

scaling = scal

print(scaling)


In [ ]:
std_vals = []

for header in headers:
    std_vals.append(data[header].std())

print(max_vals)
print(std_vals)

scaling = [std_vals[0]/std_vals[2], std_vals[0]/std_vals[4] * val_scale_scale]

print(scaling)


In [ ]:
scaled_data = data.copy()
scaled_data["fluxx"] = data["fluxx"]*scaling[0]
scaled_data["fluxy"] = data["fluxy"]*scaling[0]
scaled_data["value"] = data["value"]*scaling[1] 

# print(scaled_data)

new_max_vals = []
for header in headers:
    new_max_vals.append(scaled_data[header].max())

print(new_max_vals)


In [ ]:
pd.DataFrame(scaled_data).to_csv("data_VGQ_generated_scaled.csv", index=False)
f = open("scaling.in", "w")
f.write(str(scaling[0])+"\n")
f.write(str(scaling[1]))
f.close()

## Trimming

In [ ]:
trimmed_data = data.copy()

trimmed_data = trimmed_data.drop(trimmed_data[(trimmed_data.gradx < -33)].index)
trimmed_data = trimmed_data.drop(trimmed_data[(trimmed_data.gradx > 33)].index)
trimmed_data = trimmed_data.drop(trimmed_data[(trimmed_data.grady < -33)].index)
trimmed_data = trimmed_data.drop(trimmed_data[(trimmed_data.grady > 33)].index)
# trimmed_data = trimmed_data.drop(trimmed_data[(trimmed_data.gradx > -10) & (trimmed_data.gradx < -5)].index)
trimmed_data = trimmed_data.drop(trimmed_data[(trimmed_data.gradx > -10) & (trimmed_data.gradx < -5)].index)
trimmed_data = trimmed_data.drop(trimmed_data[(trimmed_data.grady > -10) & (trimmed_data.grady < -5)].index)

In [ ]:
scaled_data = trimmed_data.copy()
scaled_data["fluxx"] = trimmed_data["fluxx"]*scaling[0]
scaled_data["fluxy"] = trimmed_data["fluxy"]*scaling[0]
scaled_data["value"] = trimmed_data["value"]*scaling[1] 

In [ ]:
pd.DataFrame(scaled_data).to_csv("data_VGQ_generated_scaled_trimmed.csv", index=False)
f = open("scaling_trimmed.in", "w")
f.write(str(scaling[0])+"\n")
f.write(str(scaling[1]))
f.close()

In [ ]:
plt.scatter(trimmed_data["gradx"], trimmed_data["fluxx"], c=trimmed_data["value"], s=0.05, cmap="coolwarm")
# plt.xlabel('Gradient ' + r'$g_x$')
# plt.ylabel('Flux ' + r'$q_x$', fontsize=FS)
cbar = plt.colorbar()
# location='bottom'
# cbar.set_label('Temperature ' + r'$T$', fontsize=FS)
plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
plt.tight_layout()
plt.savefig('ana_dataset_temp_trimmed.png', dpi=300, facecolor="none", edgecolor='none')
plt.show()

In [ ]:
plt.scatter(trimmed_data["value"], trimmed_data["fluxx"], c=trimmed_data["gradx"], s=0.1, cmap="gist_ncar")
# plt.xlabel('Temperature ' + r'$T$')
# plt.ylabel('Flux ' + r'$q_x$')
cbar = plt.colorbar()
# cbar.set_label('Gradient ' + r'$g_x$')
cbar.formatter.set_powerlimits((0, 0))
plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
# plt.ylim([-1e3, 1e3])
plt.tight_layout()
plt.savefig('ana_dataset_grad_trimmed.png', dpi=300, facecolor="none", edgecolor='none')
plt.show()

In [ ]:
# std_vals = []

# for header in headers:
#     std_vals.append(data[header].std())

# print(max_vals)
# print(std_vals)

# scaling = [std_vals[0]/std_vals[2], std_vals[0]/std_vals[4] * val_scale_scale]

# print(scaling)


In [ ]:
# scaled_data = data
# scaled_data["fluxx"] = data["fluxx"]*scaling[0]
# scaled_data["fluxy"] = data["fluxy"]*scaling[0]
# scaled_data["value"] = data["value"]*scaling[1] 

# # print(scaled_data)

# new_max_vals = []
# for header in headers:
#     new_max_vals.append(scaled_data[header].max())

# print(new_max_vals)


In [ ]:
# pd.DataFrame(scaled_data).to_csv("data_VGQ_generated_scaled.csv", index=False)
# f = open("scaling.in", "w")
# f.write(str(scaling[0])+"\n")
# f.write(str(scaling[1]))
# f.close()

In [ ]:
scaled_data = scaled_data.drop(scaled_data[scaled_data.index % 2 > 0].index)


plot val grad flux with analysis results

In [ ]:
# import plotly as Plotly
# import plotly.graph_objects as go

# range = max_flux * scaling[0]

# fig = go.Figure()
# fig.add_trace(go.Scatter3d(x=scaled_data["value"],
#                            y=scaled_data["gradx"],
#                            z=scaled_data["fluxx"],
#                            name='material dataset',
#                            mode='markers',
#                            marker={
#                                'size': 2,
#                                'opacity': 0.8,
#                                'color': scaled_data["value"],
#                                'colorbar_title': "Temperature",
#                                'colorscale': "bluered"
#                            },
# ))
# # fig.add_trace(go.Scatter3d(x=far_data["value"],
# #                            y=far_data["gradx"],
# #                            z=far_data["fluxx"],
# #                            name='far away field values',
# #                            mode='markers',
# #                            marker={
# #                                'size': 1,
# #                                'opacity': 0.8,
# # },
# # ))
# # fig.add_trace(go.Scatter3d(x=not_far_data["value"],
# #                            y=not_far_data["gradx"],
# #                            z=not_far_data["fluxx"],
# #                            name='close field values',
# #                            mode='markers',
# #                            marker={
# #                                'size': 3,
# #                                'opacity': 0.8,
# # },
# # ))



# fig.update_layout(scene=dict(
#     xaxis_title='Temperature',
#     yaxis_title='Gradient',
#     zaxis_title='Flux',
#     # xaxis=dict(range=[-range, range]),
#     # yaxis=dict(range=[-range, range]),
#     # zaxis=dict(range=[-range, range])
#     ),
# #     font_family='Times New Roman',
# #     width=700,
# #     margin=dict(r=20, b=10, l=10, t=10),
# #     scene_aspectmode='cube'
#                  )

# fig.show()




plot scaled val grad flux with analysis results

In [ ]:
# import plotly as Plotly
# import plotly.graph_objects as go

# range = max_flux * scaling[0]

# fig = go.Figure()
# fig.add_trace(go.Scatter3d(x=scaled_data["value"]/scaling[1],
#                            y=scaled_data["gradx"],
#                            z=scaled_data["fluxx"]/scaling[0],
#                            mode='markers',
#                            marker={
#                                'size': 0.2,
#                                'opacity': 0.8,
# },
# ))
# fig.add_trace(go.Scatter3d(x=far_data["value"]/scaling[1],
#                            y=far_data["gradx"],
#                            z=far_data["fluxx"]/scaling[0],
#                            mode='markers',
#                            marker={
#                                'size': 5,
#                                'opacity': 0.8,
# },
# ))
# # fig.add_trace(go.Scatter3d(x=not_far_data["value"]/scaling[1],
# #                            y=not_far_data["gradx"],
# #                            z=not_far_data["fluxx"]/scaling[0],
# #                            mode='markers',
# #                            marker={
# #                                'size': 3,
# #                                'opacity': 0.8,
# # },
# # ))


# fig.update_layout(scene=dict(
#     xaxis_title='Temperature',
#     yaxis_title='Gradient',
#     zaxis_title='Flux',
#     # xaxis=dict(range=[-range, range]),
#     # yaxis=dict(range=[-range, range]),
#     # zaxis=dict(range=[-range, range])
# ),
#     font_family='Times New Roman',
#     width=700,
#     margin=dict(r=20, b=10, l=10, t=10),
#     scene_aspectmode='cube')

# fig.show()


plot grad grad flux with analysis results

In [ ]:
# import plotly as Plotly
# import plotly.graph_objects as go

# range = max_flux * scaling[0]

# fig = go.Figure()
# fig.add_trace(go.Scatter3d(x=scaled_data["grady"],
#                            y=scaled_data["gradx"],
#                            z=scaled_data["fluxx"],
#                            mode='markers',
#                            marker={
#                                'size': 0.2,
#                                'opacity': 0.8,
# },
# ))
# fig.add_trace(go.Scatter3d(x=far_data["grady"],
#                            y=far_data["gradx"],
#                            z=far_data["fluxx"],
#                            mode='markers',
#                            marker={
#                                'size': 5,
#                                'opacity': 0.8,
# },
# ))
# # fig.add_trace(go.Scatter3d(x=not_far_data["grady"],
# #                            y=not_far_data["gradx"],
# #                            z=not_far_data["fluxx"],
# #                            mode='markers',
# #                            marker={
# #                                'size': 3,
# #                                'opacity': 0.8,
# # },
# # ))


# fig.update_layout(scene=dict(
#     xaxis_title='Gradient y',
#     yaxis_title='Gradient x',
#     zaxis_title='Flux x',
#     # xaxis=dict(range=[-range, range]),
#     # yaxis=dict(range=[-range, range]),
#     # zaxis=dict(range=[-range, range])
# ),
#     font_family='Times New Roman',
#     width=700,
#     margin=dict(r=20, b=10, l=10, t=10),
#     scene_aspectmode='cube')

# fig.show()


In [ ]:
# pd.DataFrame(scaled_data).to_csv("data_VGQ_generated_scaled.csv", index=False)
# f = open("scaling.in", "w")
# f.write(str(scaling[0])+"\n")
# f.write(str(scaling[1]))
# f.close()

In [ ]:
# folder = "../octopus_ana/"
# !cp data_VGQ_generated_scaled.csv {folder}
# !cp scaling.in {folder}

copy to analysis folders

In [ ]:
# !cp data_VGQ_generated_scaled.csv ./L-shape_ana/
# !cp scaling.in ./L-shape_ana/


In [ ]:
# !cp data_VGQ_generated_scaled.csv /Users/adriana/mofem_install/testing/
# !cp scaling.in /Users/adriana/mofem_install/testing/
